In [ ]:
import pymortar
import glob
URL = "http://mortar2_mortar-server_1:5001"
# connect client to Mortar frontend server
c = pymortar.Client(URL)

In [ ]:
# query AHU, supply air temp sensor, and downstream zone
query1 = """SELECT ?ahu ?sat ?zone WHERE {
    ?ahu a brick:AHU .
    ?ahu brick:hasPart?/brick:hasPoint ?sat .
    ?sat a brick:Supply_Air_Temperature_Sensor .
    ?ahu brick:feeds+ ?zone .
    ?zone a brick:HVAC_Zone
}"""

# damper position commands in VAVs
query2 = """SELECT ?vav ?cmd WHERE {
    ?vav   a  brick:VAV .
    ?vav   brick:hasPart?/brick:hasPoint ?cmd .
    ?cmd a brick:Damper_Position_Command
}"""

# all air temperature sensors
query3 = """SELECT ?ats WHERE {
    ?ats  a  brick:Air_Temperature_Sensor .
}"""

## Querying (Brick) Metadata

In [ ]:
# execute query against all sites
# c.sparql(query)

# or against a list of sites
c.sparql(query1, sites=['bldg1'])

In [ ]:
# get a summary of which sites return results for a set of queries
c.qualify([query1, query2, query3])

In [ ]:
# name the queries (optional)
c.qualify({"damper_cmd": query2, "ahu sat": query1, "temp sensors": query3})

## Querying Timeseries Data

In [ ]:
# get SPARQL query results, stream metadata data tables as separate objects
res = c.data(query3)

In [ ]:
# sparql query results
res.metadata

In [ ]:
# stream metadata
res.streams

In [ ]:
# actual timeseries data
res.data